In [1]:
import numpy as np
import pandas as pd
%matplotlib inline 
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
import sklearn.model_selection as ms
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit

train_patient_data = pd.read_csv('./train_patient_data.csv', index_col=0)
Train = pd.read_csv('./data/Train.csv', index_col=0)

C:\Users\oyb50\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
train_patient_data.head()

,BeneID,ClaimID,Provider,ClmAdmitDiagnosisCode,DiagnosisGroupCode,AdmissionDuration,ClaimDuration,Total_ClmDiagnosisCode,Total_ClmProcedureCode,State,County,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Age,Total_ChronicCond,TotalClaimCost,PatientType,Total_Physician
0,BENE11001,CLM46614,PRV55912,7866,201,7.0,7,9,0,39,230,36000,3204,60,70,67,7,3867.0,1,1
1,BENE11001,CLM66048,PRV55907,6186,750,3.0,3,3,1,39,230,36000,3204,60,70,67,7,2023.0,1,2
2,BENE11001,CLM68358,PRV56046,29590,883,4.0,4,6,0,39,230,36000,3204,60,70,67,7,1517.0,1,2
3,BENE11011,CLM38412,PRV52405,431,067,9.0,9,9,1,1,360,5000,1068,250,320,96,6,674.0,1,3
4,BENE11014,CLM63689,PRV56614,78321,975,18.0,18,9,1,45,780,21260,2136,120,100,72,6,615.0,1,2


In [3]:
train_patient_data = train_patient_data.drop(columns=['IPAnnualReimbursementAmt', 'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt','OPAnnualDeductibleAmt'])

In [4]:
train_patient_data.head()

,BeneID,ClaimID,Provider,ClmAdmitDiagnosisCode,DiagnosisGroupCode,AdmissionDuration,ClaimDuration,Total_ClmDiagnosisCode,Total_ClmProcedureCode,State,County,Age,Total_ChronicCond,TotalClaimCost,PatientType,Total_Physician
0,BENE11001,CLM46614,PRV55912,7866,201,7.0,7,9,0,39,230,67,7,3867.0,1,1
1,BENE11001,CLM66048,PRV55907,6186,750,3.0,3,3,1,39,230,67,7,2023.0,1,2
2,BENE11001,CLM68358,PRV56046,29590,883,4.0,4,6,0,39,230,67,7,1517.0,1,2
3,BENE11011,CLM38412,PRV52405,431,067,9.0,9,9,1,1,360,96,6,674.0,1,3
4,BENE11014,CLM63689,PRV56614,78321,975,18.0,18,9,1,45,780,72,6,615.0,1,2


In [5]:
train_patient_data.shape

(558211, 16)

In [6]:
#1)BeneID – Unique # of patients per provider 
Unique_patients=train_patient_data.groupby('Provider')['BeneID'].unique().apply(lambda x: len(x)).to_frame("Unique_patients").reset_index()
Unique_patients

,Provider,Unique_patients
0,PRV51001,24
1,PRV51003,117
2,PRV51004,138
3,PRV51005,495
4,PRV51007,58
...,...,...
5405,PRV57759,24
5406,PRV57760,9
5407,PRV57761,67
5408,PRV57762,1


In [7]:
#2)	ClaimID - # of claims by provider
Claim_count = train_patient_data.groupby('Provider')['ClaimID'].agg("count").to_frame("Claim_count").reset_index()
Claim_count

,Provider,Claim_count
0,PRV51001,25
1,PRV51003,132
2,PRV51004,149
3,PRV51005,1165
4,PRV51007,72
...,...,...
5405,PRV57759,28
5406,PRV57760,22
5407,PRV57761,82
5408,PRV57762,1


In [8]:
#3) avg TotalClaimCost by provider,
Avg_TotalClaimCost = train_patient_data.groupby('Provider')['TotalClaimCost'].mean().to_frame("Avg_TotalClaimCost").reset_index()
Avg_TotalClaimCost

,Provider,Avg_TotalClaimCost
0,PRV51001,1121.080000
1,PRV51003,1368.287879
2,PRV51004,352.214765
3,PRV51005,244.300429
4,PRV51007,257.458333
...,...,...
5405,PRV57759,384.642857
5406,PRV57760,216.818182
5407,PRV57761,229.756098
5408,PRV57762,1900.000000


In [9]:
temp = train_patient_data.groupby(['Provider','BeneID'])['TotalClaimCost'].mean().reset_index()
temp.loc[temp['Provider']=='PRV51001'].mean()

TotalClaimCost    1145.083333
dtype: float64

In [10]:
#3) avg TotalClaimCost reimbursed per patient by provider
Avg_TotalClaimCost_PerPatient = train_patient_data.groupby(['Provider','BeneID'])['TotalClaimCost'].mean().reset_index().groupby('Provider')[['TotalClaimCost']].mean().reset_index()
Avg_TotalClaimCost_PerPatient = Avg_TotalClaimCost_PerPatient.rename(columns={"TotalClaimCost": "Avg_TotalClaimCost_PerPatient"})
Avg_TotalClaimCost_PerPatient

,Provider,Avg_TotalClaimCost_PerPatient
0,PRV51001,1145.083333
1,PRV51003,1382.559829
2,PRV51004,352.415459
3,PRV51005,256.429705
4,PRV51007,234.129310
...,...,...
5405,PRV57759,382.708333
5406,PRV57760,295.416667
5407,PRV57761,208.619403
5408,PRV57762,1900.000000


In [11]:
#4)Physician – avg # of physicians by provider
Avg_Total_Physician = train_patient_data.groupby('Provider')[['Total_Physician']].mean().reset_index()
Avg_Total_Physician = Avg_Total_Physician.rename(columns={"Total_Physician": "Avg_Total_Physician"})
Avg_Total_Physician

,Provider,Avg_Total_Physician
0,PRV51001,1.600000
1,PRV51003,1.530303
2,PRV51004,1.604027
3,PRV51005,1.599142
4,PRV51007,1.527778
...,...,...
5405,PRV57759,1.464286
5406,PRV57760,1.681818
5407,PRV57761,1.609756
5408,PRV57762,1.000000


In [12]:
# 5)ClmAdmitDiagnosisCode - # unique ClmAdmitDiagnosisCode 
Unique_ClmAdmitDiagnosisCode = train_patient_data.groupby('Provider')['ClmAdmitDiagnosisCode'].unique().apply(lambda x: len(x)).to_frame("Unique_ClmAdmitDiagnosisCode").reset_index()
Unique_ClmAdmitDiagnosisCode

,Provider,Unique_ClmAdmitDiagnosisCode
0,PRV51001,12
1,PRV51003,59
2,PRV51004,26
3,PRV51005,159
4,PRV51007,18
...,...,...
5405,PRV57759,4
5406,PRV57760,8
5407,PRV57761,17
5408,PRV57762,1


In [13]:
# 5) count of ClmAdmitDiagnosisCode per provider (sum)
ClmAdmitDiagnosisCode_count = train_patient_data.groupby('Provider')['ClmAdmitDiagnosisCode'].agg("count").to_frame("ClmAdmitDiagnosisCode_count").reset_index()
ClmAdmitDiagnosisCode_count

,Provider,ClmAdmitDiagnosisCode_count
0,PRV51001,25
1,PRV51003,132
2,PRV51004,149
3,PRV51005,1165
4,PRV51007,72
...,...,...
5405,PRV57759,28
5406,PRV57760,22
5407,PRV57761,82
5408,PRV57762,1


Marked the missing values as 'None' in the DiagnosisGroupCode 

In [14]:
# 6) DiagnosisGroupCode - # unique group code by provider
Unique_DiagnosisGroupCode = train_patient_data.groupby('Provider')[['DiagnosisGroupCode']].nunique(dropna=True).reset_index()
Unique_DiagnosisGroupCode = Unique_DiagnosisGroupCode.rename(columns={"DiagnosisGroupCode": "Unique_DiagnosisGroupCode"})
Unique_DiagnosisGroupCode

,Provider,Unique_DiagnosisGroupCode
0,PRV51001,5
1,PRV51003,58
2,PRV51004,0
3,PRV51005,0
4,PRV51007,3
...,...,...
5405,PRV57759,0
5406,PRV57760,0
5407,PRV57761,0
5408,PRV57762,0


In [15]:
# 6) DiagnosisGroupCode total # of group codes used by provider. 
DiagnosisGroupCode_count = train_patient_data.groupby('Provider')['DiagnosisGroupCode'].agg("count").to_frame("DiagnosisGroupCode_count").reset_index()
DiagnosisGroupCode_count

,Provider,DiagnosisGroupCode_count
0,PRV51001,5
1,PRV51003,62
2,PRV51004,0
3,PRV51005,0
4,PRV51007,3
...,...,...
5405,PRV57759,0
5406,PRV57760,0
5407,PRV57761,0
5408,PRV57762,0


In [16]:
# 7)TotalClmDiagnosisCode -  avg # of total diagnosis codes per claim by provider 
Avg_Total_ClmDiagnosisCode = train_patient_data.groupby('Provider')[['Total_ClmDiagnosisCode']].mean().reset_index()
Avg_Total_ClmDiagnosisCode = Avg_Total_ClmDiagnosisCode.rename(columns={"Total_ClmDiagnosisCode": "Avg_Total_ClmDiagnosisCode"})
Avg_Total_ClmDiagnosisCode

,Provider,Avg_Total_ClmDiagnosisCode
0,PRV51001,3.200000
1,PRV51003,5.250000
2,PRV51004,2.583893
3,PRV51005,2.588841
4,PRV51007,2.986111
...,...,...
5405,PRV57759,2.071429
5406,PRV57760,2.363636
5407,PRV57761,2.670732
5408,PRV57762,1.000000


In [17]:
train_patient_data.loc[train_patient_data['Provider']=='PRV51001'][['Total_ClmDiagnosisCode']].mean()

Total_ClmDiagnosisCode    3.2
dtype: float64

In [18]:
# 8)TotalClmProcedureCode - avg # of total procedure codes per claim by provider
Avg_Total_ClmProcedureCode=train_patient_data.groupby('Provider')[['Total_ClmProcedureCode']].mean().reset_index()
Avg_Total_ClmProcedureCode = Avg_Total_ClmProcedureCode.rename(columns={"Total_ClmProcedureCode": "Avg_Total_ClmProcedureCode"})
Avg_Total_ClmProcedureCode

,Provider,Avg_Total_ClmProcedureCode
0,PRV51001,0.120000
1,PRV51003,0.363636
2,PRV51004,0.000000
3,PRV51005,0.000000
4,PRV51007,0.013889
...,...,...
5405,PRV57759,0.000000
5406,PRV57760,0.000000
5407,PRV57761,0.000000
5408,PRV57762,0.000000


In [19]:
# 9)ClaimDuration/Days – avg # of claim duration by provider 
Avg_ClaimDuration = train_patient_data.groupby('Provider')[['ClaimDuration']].mean().reset_index()
Avg_ClaimDuration = Avg_ClaimDuration.rename(columns={"ClaimDuration": "Avg_ClaimDuration"})
Avg_ClaimDuration

,Provider,Avg_ClaimDuration
0,PRV51001,2.440000
1,PRV51003,4.674242
2,PRV51004,2.429530
3,PRV51005,2.088412
4,PRV51007,1.958333
...,...,...
5405,PRV57759,3.142857
5406,PRV57760,1.318182
5407,PRV57761,2.390244
5408,PRV57762,1.000000


In [20]:
# 10) Admission Duration – avg # of hospital days per claim by provider
Avg_AdmissionDuration=train_patient_data.groupby('Provider')[['AdmissionDuration']].mean().reset_index()
Avg_AdmissionDuration = Avg_AdmissionDuration.rename(columns={"AdmissionDuration": "Avg_AdmissionDuration"})
Avg_AdmissionDuration

,Provider,Avg_AdmissionDuration
0,PRV51001,6.000000
1,PRV51003,6.161290
2,PRV51004,NaN
3,PRV51005,NaN
4,PRV51007,6.333333
...,...,...
5405,PRV57759,NaN
5406,PRV57760,NaN
5407,PRV57761,NaN
5408,PRV57762,NaN


In [21]:
values = {'AdmissionDuration':0}
train_patient_data = train_patient_data.fillna(value=values)
train_patient_data[['AdmissionDuration']].isnull().sum()

AdmissionDuration    0
dtype: int64

In [22]:
# 12) Number of unique states per provider (proxy for coverage – multicollinear)
Unique_States=train_patient_data.groupby('Provider')['State'].unique().apply(lambda x:len(x)).to_frame("Unique_States").reset_index()
Unique_States

,Provider,Unique_States
0,PRV51001,1
1,PRV51003,3
2,PRV51004,9
3,PRV51005,4
4,PRV51007,2
...,...,...
5405,PRV57759,1
5406,PRV57760,1
5407,PRV57761,1
5408,PRV57762,1


In [23]:
# 12) Number of unique county per provider (proxy for coverage – multicollinear)
Unique_County=train_patient_data.groupby('Provider')['County'].unique().apply(lambda x:len(x)).to_frame("Unique_County").reset_index()
Unique_County

,Provider,Unique_County
0,PRV51001,7
1,PRV51003,23
2,PRV51004,28
3,PRV51005,26
4,PRV51007,6
...,...,...
5405,PRV57759,2
5406,PRV57760,3
5407,PRV57761,2
5408,PRV57762,1


In [24]:
# 13) Age – Avg claim age/age-bin by provider.
Avg_Age=train_patient_data.groupby('Provider')[['Age']].mean().reset_index()
Avg_Age = Avg_Age.rename(columns={"Age": "Avg_Age"})
Avg_Age

,Provider,Avg_Age
0,PRV51001,78.880000
1,PRV51003,70.083333
2,PRV51004,72.248322
3,PRV51005,70.543348
4,PRV51007,69.416667
...,...,...
5405,PRV57759,74.035714
5406,PRV57760,61.272727
5407,PRV57761,71.987805
5408,PRV57762,68.000000


In [25]:
train_patient_data.head()

,BeneID,ClaimID,Provider,ClmAdmitDiagnosisCode,DiagnosisGroupCode,AdmissionDuration,ClaimDuration,Total_ClmDiagnosisCode,Total_ClmProcedureCode,State,County,Age,Total_ChronicCond,TotalClaimCost,PatientType,Total_Physician
0,BENE11001,CLM46614,PRV55912,7866,201,7.0,7,9,0,39,230,67,7,3867.0,1,1
1,BENE11001,CLM66048,PRV55907,6186,750,3.0,3,3,1,39,230,67,7,2023.0,1,2
2,BENE11001,CLM68358,PRV56046,29590,883,4.0,4,6,0,39,230,67,7,1517.0,1,2
3,BENE11011,CLM38412,PRV52405,431,067,9.0,9,9,1,1,360,96,6,674.0,1,3
4,BENE11014,CLM63689,PRV56614,78321,975,18.0,18,9,1,45,780,72,6,615.0,1,2


In [26]:
#14) ChronicCondition – avg # of total ChronicConditions per claim by provider 
Avg_Total_ChronicCond=train_patient_data.groupby('Provider')[['Total_ChronicCond']].mean().reset_index()
Avg_Total_ChronicCond = Avg_Total_ChronicCond.rename(columns={"Total_ChronicCond": "Avg_Total_ChronicCond"})
Avg_Total_ChronicCond

,Provider,Avg_Total_ChronicCond
0,PRV51001,5.880000
1,PRV51003,4.765152
2,PRV51004,4.496644
3,PRV51005,4.557940
4,PRV51007,4.319444
...,...,...
5405,PRV57759,5.428571
5406,PRV57760,3.500000
5407,PRV57761,5.121951
5408,PRV57762,5.000000


In [27]:
data_frames = [Unique_patients,Claim_count,Avg_TotalClaimCost_PerPatient,Avg_TotalClaimCost,
               Avg_Total_Physician,Unique_ClmAdmitDiagnosisCode,ClmAdmitDiagnosisCode_count,
               Unique_DiagnosisGroupCode,DiagnosisGroupCode_count,Avg_Total_ClmDiagnosisCode,
               Avg_Total_ClmProcedureCode,Avg_ClaimDuration,Avg_AdmissionDuration, Unique_States,
               Unique_County, Avg_Age, Avg_Total_ChronicCond]

In [28]:
from functools import reduce

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Provider'],
                                            how='outer'), data_frames).fillna(0)

In [29]:
df_merged

,Provider,Unique_patients,Claim_count,Avg_TotalClaimCost_PerPatient,Avg_TotalClaimCost,Avg_Total_Physician,Unique_ClmAdmitDiagnosisCode,ClmAdmitDiagnosisCode_count,Unique_DiagnosisGroupCode,DiagnosisGroupCode_count,Avg_Total_ClmDiagnosisCode,Avg_Total_ClmProcedureCode,Avg_ClaimDuration,Avg_AdmissionDuration,Unique_States,Unique_County,Avg_Age,Avg_Total_ChronicCond
0,PRV51001,24,25,1145.083333,1121.080000,1.600000,12,25,5,5,3.200000,0.120000,2.440000,6.000000,1,7,78.880000,5.880000
1,PRV51003,117,132,1382.559829,1368.287879,1.530303,59,132,58,62,5.250000,0.363636,4.674242,6.161290,3,23,70.083333,4.765152
2,PRV51004,138,149,352.415459,352.214765,1.604027,26,149,0,0,2.583893,0.000000,2.429530,0.000000,9,28,72.248322,4.496644
3,PRV51005,495,1165,256.429705,244.300429,1.599142,159,1165,0,0,2.588841,0.000000,2.088412,0.000000,4,26,70.543348,4.557940
4,PRV51007,58,72,234.129310,257.458333,1.527778,18,72,3,3,2.986111,0.013889,1.958333,6.333333,2,6,69.416667,4.319444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,24,28,382.708333,384.642857,1.464286,4,28,0,0,2.071429,0.000000,3.142857,0.000000,1,2,74.035714,5.428571
5406,PRV57760,9,22,295.416667,216.818182,1.681818,8,22,0,0,2.363636,0.000000,1.318182,0.000000,1,3,61.272727,3.500000
5407,PRV57761,67,82,208.619403,229.756098,1.609756,17,82,0,0,2.670732,0.000000,2.390244,0.000000,1,2,71.987805,5.121951
5408,PRV57762,1,1,1900.000000,1900.000000,1.000000,1,1,0,0,1.000000,0.000000,1.000000,0.000000,1,1,68.000000,5.000000


In [30]:
len(data_frames) + 1

18

In [31]:
Train.head()

,PotentialFraud
Provider,
PRV51001,No
PRV51003,Yes
PRV51004,No
PRV51005,Yes
PRV51007,No


In [32]:
final1 =df_merged.merge(Train, how='inner', on='Provider')
final1.head()

,Provider,Unique_patients,Claim_count,Avg_TotalClaimCost_PerPatient,Avg_TotalClaimCost,Avg_Total_Physician,Unique_ClmAdmitDiagnosisCode,ClmAdmitDiagnosisCode_count,Unique_DiagnosisGroupCode,DiagnosisGroupCode_count,Avg_Total_ClmDiagnosisCode,Avg_Total_ClmProcedureCode,Avg_ClaimDuration,Avg_AdmissionDuration,Unique_States,Unique_County,Avg_Age,Avg_Total_ChronicCond,PotentialFraud
0,PRV51001,24,25,1145.083333,1121.080000,1.600000,12,25,5,5,3.200000,0.120000,2.440000,6.000000,1,7,78.880000,5.880000,No
1,PRV51003,117,132,1382.559829,1368.287879,1.530303,59,132,58,62,5.250000,0.363636,4.674242,6.161290,3,23,70.083333,4.765152,Yes
2,PRV51004,138,149,352.415459,352.214765,1.604027,26,149,0,0,2.583893,0.000000,2.429530,0.000000,9,28,72.248322,4.496644,No
3,PRV51005,495,1165,256.429705,244.300429,1.599142,159,1165,0,0,2.588841,0.000000,2.088412,0.000000,4,26,70.543348,4.557940,Yes
4,PRV51007,58,72,234.129310,257.458333,1.527778,18,72,3,3,2.986111,0.013889,1.958333,6.333333,2,6,69.416667,4.319444,No


In [33]:
final1['PotentialFraud']= final1['PotentialFraud'].replace('No', 0)
final1['PotentialFraud']= final1['PotentialFraud'].replace('Yes', 1)
final1

,Provider,Unique_patients,Claim_count,Avg_TotalClaimCost_PerPatient,Avg_TotalClaimCost,Avg_Total_Physician,Unique_ClmAdmitDiagnosisCode,ClmAdmitDiagnosisCode_count,Unique_DiagnosisGroupCode,DiagnosisGroupCode_count,Avg_Total_ClmDiagnosisCode,Avg_Total_ClmProcedureCode,Avg_ClaimDuration,Avg_AdmissionDuration,Unique_States,Unique_County,Avg_Age,Avg_Total_ChronicCond,PotentialFraud
0,PRV51001,24,25,1145.083333,1121.080000,1.600000,12,25,5,5,3.200000,0.120000,2.440000,6.000000,1,7,78.880000,5.880000,0
1,PRV51003,117,132,1382.559829,1368.287879,1.530303,59,132,58,62,5.250000,0.363636,4.674242,6.161290,3,23,70.083333,4.765152,1
2,PRV51004,138,149,352.415459,352.214765,1.604027,26,149,0,0,2.583893,0.000000,2.429530,0.000000,9,28,72.248322,4.496644,0
3,PRV51005,495,1165,256.429705,244.300429,1.599142,159,1165,0,0,2.588841,0.000000,2.088412,0.000000,4,26,70.543348,4.557940,1
4,PRV51007,58,72,234.129310,257.458333,1.527778,18,72,3,3,2.986111,0.013889,1.958333,6.333333,2,6,69.416667,4.319444,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,24,28,382.708333,384.642857,1.464286,4,28,0,0,2.071429,0.000000,3.142857,0.000000,1,2,74.035714,5.428571,0
5406,PRV57760,9,22,295.416667,216.818182,1.681818,8,22,0,0,2.363636,0.000000,1.318182,0.000000,1,3,61.272727,3.500000,0
5407,PRV57761,67,82,208.619403,229.756098,1.609756,17,82,0,0,2.670732,0.000000,2.390244,0.000000,1,2,71.987805,5.121951,0
5408,PRV57762,1,1,1900.000000,1900.000000,1.000000,1,1,0,0,1.000000,0.000000,1.000000,0.000000,1,1,68.000000,5.000000,0


In [34]:
final1.PotentialFraud.value_counts()

0    4904
1     506
Name: PotentialFraud, dtype: int64

In [37]:
y = final1.PotentialFraud
X = final1.drop(['PotentialFraud', 'Provider'], axis=1)
lr=LogisticRegression()
clf_1 = lr.fit(X, y)
clf_1.score(X,y)

C:\Users\oyb50\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9316081330868762

In [36]:
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression

# Separate majority and minority classes
final1_majority = final1[final1.PotentialFraud==0]
final1_minority = final1[final1.PotentialFraud==1]
 
# Upsample minority class
final1_minority_upsampled = resample(final1_minority, replace=True, n_samples=4904, random_state=123)
 
# Combine majority class with upsampled minority class
final1_upsampled = pd.concat([final1_majority, final1_minority_upsampled])
 
# Display new class counts
final1_upsampled.PotentialFraud.value_counts()
# 1    576
# 0    576
# Name: balance, dtype: int64
#As you can see, the new DataFrame
# Separate input features (X) and target variable (y)
y = final1_upsampled.PotentialFraud
X = final1_upsampled.drop(['PotentialFraud', 'Provider'], axis=1)
 
# Train model
logit = LogisticRegression()
clf_1 = logit.fit(X, y)
 
# Predict on training set
pred_y_1 = clf_1.predict(X)
 
clf_1.score(X, y)

C:\Users\oyb50\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8752039151712887

In [38]:
final1_upsampled

,Provider,Unique_patients,Claim_count,Avg_TotalClaimCost_PerPatient,Avg_TotalClaimCost,Avg_Total_Physician,Unique_ClmAdmitDiagnosisCode,ClmAdmitDiagnosisCode_count,Unique_DiagnosisGroupCode,DiagnosisGroupCode_count,Avg_Total_ClmDiagnosisCode,Avg_Total_ClmProcedureCode,Avg_ClaimDuration,Avg_AdmissionDuration,Unique_States,Unique_County,Avg_Age,Avg_Total_ChronicCond,PotentialFraud
0,PRV51001,24,25,1145.083333,1121.080000,1.600000,12,25,5,5,3.200000,0.120000,2.440000,6.000000,1,7,78.880000,5.880000,0
2,PRV51004,138,149,352.415459,352.214765,1.604027,26,149,0,0,2.583893,0.000000,2.429530,0.000000,9,28,72.248322,4.496644,0
4,PRV51007,58,72,234.129310,257.458333,1.527778,18,72,3,3,2.986111,0.013889,1.958333,6.333333,2,6,69.416667,4.319444,0
5,PRV51008,36,43,423.666667,399.348837,1.534884,10,43,2,2,3.162791,0.046512,2.534884,5.000000,2,9,75.651163,4.139535,0
6,PRV51011,53,58,223.132075,223.120690,1.568966,14,58,1,1,2.534483,0.000000,1.551724,6.000000,1,9,74.448276,4.793103,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2364,PRV53951,49,54,1520.255102,1487.629630,1.703704,22,54,21,22,4.537037,0.203704,3.055556,5.136364,3,15,75.759259,5.092593,1
4939,PRV57191,784,1500,554.155085,541.870000,1.229333,264,1500,158,185,3.275333,0.101333,2.932667,7.091892,16,44,73.671333,4.521333,1
314,PRV51399,199,211,1162.391122,1388.412322,1.573460,78,211,61,65,4.341232,0.232227,3.516588,6.861538,9,23,72.900474,4.924171,1
2524,PRV54144,103,294,318.647943,251.564626,1.540816,54,294,0,0,2.340136,0.000000,2.350340,0.000000,2,5,74.119048,4.622449,1


In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(final1_upsampled.drop(['PotentialFraud', 'Provider'], axis=1),final1_upsampled.PotentialFraud, test_size=0.3, random_state=1)

In [49]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression(solver='liblinear', max_iter=) #,'newton-cg', 'sag', 'saga', max_iter 1000, 2500, 5000]

logistic.fit(X_train, y_train)
print("Train Score", logistic.score(X_train, y_train))
print("Test Score", logistic.score(X_test, y_test))

Train Score 0.8780772032046613
Test Score 0.8705402650356778


C:\Users\oyb50\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [50]:
logistic.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [67]:
param_grid = [
    {'penalty':['l1', 'l2', 'elasticnet'],
    'C': np.logspace(-4, 4, 20),
    'solver': ['liblinear']
    'max_iter': [100]
    }
]

In [68]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(logistic, param_grid = param_grid, cv=5, verbose=True, n_jobs=-1, scoring='roc_auc')

In [69]:
best_clf= clf.fit(X_train,y_train)

Fitting 5 folds for each of 1200 candidates, totalling 6000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 290 tasks      | elapsed:   42.5s
[Parallel(n_jobs=-1)]: Done 629 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 1125 tasks      | elapsed:  2.9min


KeyboardInterrupt: 

In [62]:
# Best Parameters
best_clf.best_estimator_

LogisticRegression(C=0.615848211066026, max_iter=2500)

In [63]:
# Best GridSearch/CV score 
best_clf.score(X_train,y_train)

0.8766205389657684

In [64]:
best_clf.score(X_test,y_test)

0.8657832144070676